In [1]:
from functools import cmp_to_key, lru_cache, total_ordering, partial, partialmethod, reduce, singledispatch, update_wrapper, wraps

## functools.cmp_to_key(func)
----
Transform an old-style comparison function to a key function. Used with tools that accept key functions (such as sorted(), min(), max(), heapq.nlargest(), heapq.nsmallest(), itertools.groupby()). This function is primarily used as a transition tool for programs being converted from Python 2 which supported the use of comparison functions.

A comparison function is any callable that accept two arguments, compares them, and returns a negative number for less-than, zero for equality, or a positive number for greater-than. A key function is a callable that accepts one argument and returns another value to be used as the sort key.

> python2 에서 쓰이던 cmp function을 key function으로 바꾸는 역할을 한다.

In [23]:
def fake_print(some):
    print(some, end = ' ')

In [24]:
def cmp_to_key(mycmp):
    'Convert a cmp= funciton into a key= function'
    class K:
        def __init__(self, obj, *args):
            fake_print(obj)
            self.obj = obj
        def __lt__(self, other):
            fake_print("__lt__")
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            fake_print("__gt__")
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            fake_print("__eq__")
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            fake_print("__le__")
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            fake_print("__ge__")
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            fake_print("__ne__")
            return mycmp(self.obj, other.obj) != 0
    return K

In [25]:
def reverse_numeric(x, y):
    return y - x

def numeric_compare(x, y):
    return x - y

In [27]:
print(sorted([5,2,4,1,3,1], key = cmp_to_key(reverse_numeric))) # 버블 정렬처럼 비교해서 sort 하는거 같음.. 
print(sorted([5,2,4,1,3], key = cmp_to_key(numeric_compare))) # 안 쓸거 같으니깐 패스

5 2 4 1 3 1 __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ [5, 4, 3, 2, 1, 1]
5 2 4 1 3 __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ [1, 2, 3, 4, 5]


In [32]:
def custom(x, y):
    return x*x - y*y
print(sorted([5,2,4,1,3], key = cmp_to_key(custom)))

5 2 4 1 3 __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ __lt__ [1, 2, 3, 4, 5]


## @functools.lru_cache(maxsize=128, typed=False)
### lru (least recently used)
- - -
Decorator to wrap a function with a memoizing callable that saves up to the maxsize most recent calls. It can save time when an expensive or I/O bound function is periodically called with the same arguments.

Since a dictionary is used to cache results, the positional and keyword arguments to the function must be hashable.

Distinct argument patterns may be considered to be distinct calls with separate cache entries. For example, f(a=1, b=2) and f(b=2, a=1) differ in their keyword argument order and may have two separate cache entries.

If maxsize is set to None, the LRU feature is disabled and the cache can grow without bound. The LRU feature performs best when maxsize is a power-of-two.

If typed is set to true, function arguments of different types will be cached separately. For example, f(3) and f(3.0) will be treated as distinct calls with distinct results.

To help measure the effectiveness of the cache and tune the maxsize parameter, the wrapped function is instrumented with a cache_info() function that returns a named tuple showing hits, misses, maxsize and currsize. In a multi-threaded environment, the hits and misses are approximate.

The decorator also provides a cache_clear() function for clearing or invalidating the cache.

The original underlying function is accessible through the __wrapped__ attribute. This is useful for introspection, for bypassing the cache, or for rewrapping the function with a different cache.

An LRU (least recently used) cache works best when the most recent calls are the best predictors of upcoming calls (for example, the most popular articles on a news server tend to change each day). The cache’s size limit assures that the cache does not grow without bound on long-running processes such as web servers.

In general, the LRU cache should only be used when you want to reuse previously computed values. Accordingly, it doesn’t make sense to cache functions with side-effects, functions that need to create distinct mutable objects on each call, or impure functions such as time() or random().

##### 페이지 교체 알고리즘으로 배웠던 lru 이다. 
##### 그래서 오랜만에 다시 복습
> LRU (Least Recently Used) : 최근에 사용하지 않은 페이지를 교체 <br>
LFU (Least Frequently Used) : 사용 횟수가 가장 적은 페이지를 교체 <br>
NUR (Not Used Recently) : 최근에 사용하지 않은 페이지를 교체 <br>
FIFO(Fisrt In First Out) : 먼저 적재한 페이지부터 교체 <br>
MFU(Most Frequently Used) : 사용 횟수가 가장 많은 페이지를 교체 <br>
OPT(OPTimal replacement) : 가장 오랫동안 사용하지 않을 것으로 예측한 페이지를 교체 (이상적이지만 구현할 수 없음.) <br>
SCR(Second Chance Replacement) : FIFO 기법의 단점을 보완하는 기법으로 교체 대상을 판별하기 전에 참조 비트를 검사하여 1일 때 한 번의 기회를 더 부여
참조 비트가 1이면 큐의 맨 뒤로 피드백.


In [34]:
import urllib
@lru_cache(maxsize=32)
def get_pep(num):
    'Retrieve text of a Python Enghancement Proposal'
    resource = 'http://www.python.org/dev/peps/pep-%04d/' % num
    try:
        with urllib.request.urlopen(resource) as s:
            return s.read()
    except urllib.error.HTTPError:
        return 'Not Found'

In [35]:
for n in 8, 290, 308, 320, 8, 218, 320, 279, 289, 320, 9991:
    pep = get_pep(n)
    print(n, len(pep))

8 105879
290 59822
308 57028
320 49607
8 105879
218 46851
320 49607
279 48609
289 50938
320 49607
9991 9


In [37]:
get_pep.cache_info()

CacheInfo(hits=3, misses=8, maxsize=32, currsize=8)

In [69]:
@lru_cache(maxsize=3)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [70]:
[fib(n) for n in range(16)] # 와... 역시 cache
# fibonacci 에서는 lfu 가 더 좋지 않을까... 생각만 해봅니다.
# 다시 생각해보니 dp 처럼 작동해서 lru가 최고 일 듯

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

In [71]:
print(fib.cache_info())
"""설명
cache hit: 참조하려는 데이터가 캐시에 존재할 때 캐시 히트라 한다.
cache miss: 참조하려는 데이터가 캐시에 존재 하지 않을 때 캐시 미스라 한다.
cache hit ratio : 적중률 = (캐시히트횟수) / (전체참조횟수)
"""

CacheInfo(hits=28, misses=16, maxsize=3, currsize=3)


'설명\ncache hit: 참조하려는 데이터가 캐시에 존재할 때 캐시 히트라 한다.\ncache miss: 참조하려는 데이터가 캐시에 존재 하지 않을 때 캐시 미스라 한다.\ncache hit ratio : 적중률 = (캐시히트횟수) / (전체참조횟수)\n'

## @functools.total_ordering
- - -
Given a class defining one or more rich comparison ordering methods, this class decorator supplies the rest. This simplifies the effort involved in specifying all of the possible rich comparison operations:

The class must define one of __lt__(), __le__(), __gt__(), or __ge__(). In addition, the class should supply an __eq__() method.
> eq() 방법과 , lt(), le(), ge(), gt() 중 한 가지를 작성하면 나머지 방법을 자동으로 작성해준다.

In [99]:
@total_ordering
class Student:
    def __init__(self, lastname, firstname):
        self.lastname = lastname
        self.firstname = firstname
    def __eq__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) == (other.lastname.lower(), other.firstname.lower()))
    def __lt__(self, other):
        return ((self.lastname.lower(), self.firstname.lower()) < (other.lastname.lower(), other.firstname.lower()))

In [100]:
st1 = Student("gunmo","Goo")
st2 = Student("jina","baek")

In [102]:
print(st1 == st2)
print(st1 == st1)
print(st1 > st2)
print(st1 < st2)
print(st1 >= st2)
print(st1 <= st2)

False
True
False
True
False
True


## functools.partial(func, *args, **keywords)

Return a new partial object which when called will behave like func called with the positional arguments args and keyword arguments keywords. If more arguments are supplied to the call, they are appended to args. If additional keyword arguments are supplied, they extend and override keywords. Roughly equivalent to:
> partial object를 돌려주는 함수. partial object는 args와 keywords를 가지고 함수처럼 작동합니다.<br>
args가 추가되면 더하고 keywords가 추가되면 재생성한다.

```python
def partial(func, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = keywords.copy()
        newkeywords.update(fkeywords)
        return func(*args, *fargs, **newkeywords)
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc
```

The partial() is used for partial function application which “freezes” some portion of a function’s arguments and/or keywords resulting in a new object with a simplified signature. For example, partial() can be used to create a callable that behaves like the int() function where the base argument defaults to two:

##### partial object
> partial objects are callable objects created by partial(). They have three read-only attributes: <br>
partial.func :
A callable object or function. Calls to the partial object will be forwarded to func with new arguments and keywords.
<br>partial.args :
The leftmost positional arguments that will be prepended to the positional arguments provided to a partial object call.
<br>partial.keywords :
The keyword arguments that will be supplied when the partial object is called.

In [104]:
basetwo = partial(int, base = 2)

In [107]:
print(basetwo.func)
print(basetwo.args)
print(basetwo.keywords)

<class 'int'>
()
{'base': 2}


In [109]:
basetwo.__doc__ = 'Convert base 2 string to an int.'
basetwo('10010')

18

In [113]:
int('10010', base = 2)

18

In [115]:
basetwo('10010', base = 3)

84

## class functools.partialmethod(func, *args, **keywords)

Return a new partialmethod descriptor which behaves like partial except that it is designed to be used as a method definition rather than being directly callable.

func must be a descriptor or a callable (objects which are both, like normal functions, are handled as descriptors).

When func is a descriptor (such as a normal Python function, classmethod(), staticmethod(), abstractmethod() or another instance of partialmethod), calls to __get__ are delegated to the underlying descriptor, and an appropriate partial object returned as the result.

When func is a non-descriptor callable, an appropriate bound method is created dynamically. This behaves like a normal Python function when used as a method: the self argument will be inserted as the first positional argument, even before the args and keywords supplied to the partialmethod constructor.

> descriptor : class임

In [126]:
class Cell(object):
    def __init__(self):
        self._alive = False
        
    @property
    def alive(self):
        return self._alive

    def set_state(self, state):
        self._alive = bool(state)
    
    set_alive = partialmethod(set_state, True)
    set_a = partial(set_state, state = True)
    set_dead = partialmethod(set_state, False)

In [127]:
c = Cell()
c.alive

False

In [128]:
c.set_a() 

In [129]:
c.alive

True

## functools.reduce(function, iterable[, initializer])

Apply function of two arguments cumulatively to the items of sequence, from left to right, so as to reduce the sequence to a single value. For example, reduce(lambda x, y: x+y, [1, 2, 3, 4, 5]) calculates ((((1+2)+3)+4)+5). The left argument, x, is the accumulated value and the right argument, y, is the update value from the sequence. If the optional initializer is present, it is placed before the items of the sequence in the calculation, and serves as a default when the sequence is empty. If initializer is not given and sequence contains only one item, the first item is returned.

Roughly equivalent to:
```python
def reduce(function, iterable, initializer=None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value
```

## @functools.singledispatch

Transform a function into a single-dispatch generic function.

To define a generic function, decorate it with the @singledispatch decorator. Note that the dispatch happens on the type of the first argument, create your function accordingly:

> 파이썬에서 제네릭 함수를 정의하게 해준다. <br><br>

##### 제네릭 함수

> 어떤 하나의 함수 (혹은 겉으로 보기에 이름이 다른 여러 다른 함수)가 여러 타입의 인자를 받고, 인자의 타입에 따라 적절한 동작을 하는 함수를 제네릭 함수라고 한다. C++이나 Swift에 이런 제네릭 관련 기능이 언어 레벨에서 지원되는데, 사실 파이썬은 동적 타입 언어이기 때문에 언어수준의 명시적인 제네릭 지원 기능은 없다.

##### 추가 설명

> 제네릭(Generic)은 함수나 다른 타입에 의존하는 타입을 정의할 때, 인자나 중첩된 타입의 인자에 구애받지 않고, 같은 이름의 함수를 인자 타입에 따라 반복적으로 정의하지 않아도 되도록 하는 정적 언어의 편의 장치라고 보면 된다. 그런데 동적 언어는 선언이나 정의의 시점에 타입은 거의 결정되지 않는다. 따라서 파이썬은 제네릭이라는 문법 장치를 도입할 필요가 없다.

출처 : https://soooprmx.com/archives/5852

In [137]:
@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

To add overloaded implementations to the function, use the register() attribute of the generic function. It is a decorator. For functions annotated with types, the decorator will infer the type of the first argument automatically:

In [138]:
@fun.register
def _(arg:int, verbose=False):
    if verbose:
        print('Strength in numbers, eh?', end = " ")
    print(arg)

In [139]:
@fun.register
def _(arg:list, verbose=False):
    if verbose:
        print('Enumerate this:')
    for i, elem in enumerate(arg):
        print(i, elem)

For code which doesn’t use type annotations, the appropriate type argument can be passed explicitly to the decorator itself:

In [140]:
@fun.register(complex)
def _(arg, verbose = False):
    if verbose:
        print("Better than complicated.", end = " ")
    print(arg.real, arg.imag)

To enable registering lambdas and pre-existing functions, the register() attribute can be used in a functional form:

In [141]:
def nothing(arg, verbose = False):
    print("Nothing.")

In [142]:
fun.register(type(None), nothing)

<function __main__.nothing(arg, verbose=False)>

The register() attribute returns the undecorated function which enables decorator stacking, pickling, as well as creating unit tests for each variant independently:

In [145]:
from decimal import *
@fun.register(float)
@fun.register(Decimal)
def fun_num(arg, verbose=False):
    if verbose:
        print("Half of your number:", end = " ")
    print(arg / 2)

In [147]:
fun("Hello, world.")
fun("test.", verbose=True)
fun(42, verbose=True)
fun(['spam','spam','eggs','spam'], verbose=True)
fun(None)
fun(1.23)

Hello, world.
Let me just say, test.
Strength in numbers, eh? 42
Enumerate this:
0 spam
1 spam
2 eggs
3 spam
Nothing.
0.615


Where there is no registered implementation for a specific type, its method resolution order is used to find a more generic implementation. The original function decorated with @singledispatch is registered for the base object type, which means it is used if no better implementation is found.

To check which implementation will the generic function choose for a given type, use the dispatch() attribute:

In [149]:
print(fun.dispatch(float))
print(fun.dispatch(dict))

<function fun_num at 0x7f36f43f89d8>
<function fun at 0x7f36f43f8598>


To access all registered implementations, use the read-only registry attribute:

In [156]:
print(fun.registry.keys())
print(fun.registry[float])
print(fun.registry[object])
print(fun.registry[Decimal]) ## ?

dict_keys([<class 'object'>, <class 'int'>, <class 'list'>, <class 'complex'>, <class 'NoneType'>, <class 'decimal.Decimal'>, <class 'float'>])
<function fun_num at 0x7f36f43f89d8>
<function fun at 0x7f36f43f8598>
<function fun_num at 0x7f36f43f89d8>


## functools.update_wrapper(wrapper, wrapped, assigned = WRAPPER_ASSIGNMENTS, updated = WRAPPER_UPDATES)

Update a wrapper function to look like the wrapped function. The optional arguments are tuples to specify which attributes of the original function are assigned directly to the matching attributes on the wrapper function and which attributes of the wrapper function are updated with the corresponding attributes from the original function. The default values for these arguments are the module level constants WRAPPER_ASSIGNMENTS (which assigns to the wrapper function’s __module__, __name__, __qualname__, __annotations__ and __doc__, the documentation string) and WRAPPER_UPDATES (which updates the wrapper function’s __dict__, i.e. the instance dictionary).

To allow access to the original function for introspection and other purposes (e.g. bypassing a caching decorator such as lru_cache()), this function automatically adds a __wrapped__ attribute to the wrapper that refers to the function being wrapped.

The main intended use for this function is in decorator functions which wrap the decorated function and return the wrapper. If the wrapper function is not updated, the metadata of the returned function will reflect the wrapper definition rather than the original function definition, which is typically less than helpful.

update_wrapper() may be used with callables other than functions. Any attributes named in assigned or updated that are missing from the object being wrapped are ignored (i.e. this function will not attempt to set them on the wrapper function). AttributeError is still raised if the wrapper function itself is missing any attributes named in updated.
```
wrapper 함수를 wrapped 함수처럼 보이도록 갱신한다. optional argument는 튜플로써 원래의 함수에서 <br>

1. 원래 함수의 어떤 속성을 wrapper 함수에 매칭시킬지
2. wrapper 함수의 어떤 attributes를 원해 함수의 값으로 업데이트 할지 명시한다.
    
이 optional argument는 모듈 수준의 상수인 WRAPPER_ASSIGNMENTS 와 WAPPER_UPDATES 이다.

만약 원래의 함수를 introspection 하거나, 
다른 목적을 (예를 들어, caching decorator 인 lru_cache() 를 우회한다거나) 으로 접근을 허용하기 위해 
이 함수는 자동으로 __wrapped__ attribute 를 추가하여, 원래의 함수를 참조할 수 있도록 합니다

이 함수의 목적은 어떤 함수를 decorated 하여 wrapper 로 반환하는 데코레이터 함수내에서 사용한다. 
만약, wrapper 함수가 갱신되지 않는다면, 반환된 함수(wrapper function) 의 메타데이터는 원래의 함수의 
정의를 투영하지 않고, 그 정보는 유용하지 않을 것이다.
```
더 나아가서 파이썬의 함수 객체는 몇 가지 추가적인 attributes가 있다.

```c
// include/funcobject.h # 21
typedef struct {
  PyObject_HEAD
  PyObject *func_code;        /* A code object, the __code__ attribute */
  PyObject *func_globals;     /* A dictionary (other mappings won't do) */
  PyObject *func_defaults;    /* NULL or a tuple */
  PyObject *func_kwdefaults;  /* NULL or a dict */
  PyObject *func_closure;     /* NULL or a tuple of cell objects */
  PyObject *func_doc;         /* The __doc__ attribute, can be anything */
  PyObject *func_name;        /* The __name__ attribute, a string object */
  PyObject *func_dict;        /* The __dict__ attribute, a dict or NULL */
  PyObject *func_weakreflist; /* List of weak references */
  PyObject *func_module;      /* The __module__ attribute, can be anything */
  PyObject *func_annotations; /* Annotations, a dict or NULL */
  PyObject *func_qualname;    /* The qualified name */

  /* Invariant:
   *   func_closure contains the bindings for func_code->co_freevars, so
   *   PyTuple_Size(func_closure) == PyCode_GetNumFree(func_code)
   *   (func_closure may be NULL if PyCode_GetNumFree(func_code) == 0).
  */
} PyFunctionObject;
```

문서에 등장한 각 상수에 대해 설명하자면

> WRAPPER_ASSIGNMENTS : *func_doc , *func_name, *func_module 를 의미 <br>
WRAPPER_UPDATES : *func_dict 를 의미 <br>

출처 : https://velog.io/@doondoony/python-functools-wraps


In [31]:
def wrapper(*args, **kwargs):
    print('called wrapper')
    pass

def add(a, b):
    """ ADD a + b """
    return a + b

In [32]:
print(wrapper.__doc__) # None
print(wrapper.__name__) # wrapper
add = update_wrapper(wrapper, add) # wrapped 된 add 함수를 반환
print(wrapper.__doc__)
print(wrapper.__name__)

None
wrapper
 ADD a + b 
add


In [35]:
print(wrapper(1,2)) # wrapper 의 WRAPPER_ASSIGNMENTS, WRAPPER_UPDATES 가 업데이트 된 것을 확인할 수 있다.

called wrapper
None


## functools.wraps(wrapped, assigned=WRAPPER_ASSIGNMENTS, updated=WRAPPER_UPDATES)

This is a convenience function for invoking update_wrapper() as a function decorator when defining a wrapper function. It is equivalent to partial(update_wrapper, wrapped=wrapped, assigned=assigned, updated=updated). 

> wrapper 함수를 정의 할 때 update_wrapper()를 함수 데코레이터로써 호출하는 편리한 함수입니다.

For example:

In [2]:
def my_decorator(f):
    @wraps(f) # update_wrapper를 함수 데코레이터로써 호출
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

In [3]:
@my_decorator
def example():
    """Docstring"""
    print("Called example function")

In [4]:
example()

Calling decorated function
Called example function


In [5]:
print(example.__name__) # wraps를 쓰지 않았다면 decorator의 함수 이름이 출력된다.
print(example.__doc__) # wraps를 쓰지 않았다면 decorator의 doc string 이 출력된다.

example
Docstring


Without the use of this decorator factory, the name of the example function would have been 'wrapper', and the docstring of the original example() would have been lost.

In [39]:
def adder(a, b, c, *args): return a + b + c + sum(args)

In [40]:
new_adder = partial(adder, 1,2,3,4,5)

In [41]:
new_adder(4, 5, 6)

30

```python
functools.wraps(func) == partial(update_wrapper, wrapper =func)
```
functools.wraps(func) 은 update_wrapper 함수를 적용 wrapper 에다가 func 넘겨줌